__Author:__ Bram Van de Sande

__Date:__ 11 FEB 2018

__Outline:__ This notebook clarifies the process by which the co-expression modules derived from GENIE3 can be refined into true regulomes (i.e. excluding indirect targets of transcription factors). Aka "RcisTarget".

In [1]:
import os
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyscenic.rnkdb import FeatherRankingDatabase as RankingDatabase, SQLiteRankingDatabase, MemoryDecorator
from pyscenic.genesig import GeneSignature, Regulome
from pyscenic.regulome import module2features_bincount_impl, derive_regulomes, module2features_numba_impl, module2regulome, module2df
from pyscenic.utils import load_motif_annotations

from dask import delayed
from dask.dot import dot_graph
from dask.multiprocessing import get
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from dask.diagnostics import ProgressBar
from distributed import LocalCluster, Client
from bokeh.io import output_notebook, push_notebook, show
output_notebook()
from dask.diagnostics import visualize

Loading BokehJS ...

In [2]:
%load_ext snakeviz
%load_ext line_profiler

In [3]:
DATA_FOLDER="/Users/bramvandesande/Projects/lcb/tmp"
RESOURCES_FOLDER="/Users/bramvandesande/Projects/lcb/resources"
DATABASE_FOLDER = "/Users/bramvandesande/Projects/lcb/databases/"

SQLITE_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*mc8nr.db")
FEATHER_GLOB = os.path.join(DATABASE_FOLDER, "mm9-*mc8nr.feather")

MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDER, "motifs-v9-nr.mgi-m0.001-o0.0.tbl")

NOMENCLATURE = "MGI"

Make databases in feather format are available.

In [4]:
if False:
    def derive_db_name(fname):
        return os.path.basename(fname).split(".")[0]

    from pyscenic.rnkdb import convert2feather
    
    for fname in glob.glob(SQLITE_GLOB):
        convert2feather(fname, DATABASE_FOLDER, derive_db_name(fname), NOMENCLATURE)

### Load resources

Co-expression modules were derived from GENIE3 output.

In [5]:
with open(os.path.join(DATA_FOLDER,'modules.pickle'), 'rb') as f:
    modules = pickle.load(f)

In [6]:
len(modules)

5106

### Load whole genome ranking databases

All implementations of the database are loaded for performance testing.

In [7]:
def name(fname):
    return os.path.basename(fname).split(".")[0]

In [8]:
db_fnames = glob.glob(FEATHER_GLOB)
dbs = [RankingDatabase(fname=fname, name=name(fname), nomenclature="MGI") for fname in db_fnames]

In [9]:
len(dbs)

6

In [10]:
sqldb_fnames = glob.glob(SQLITE_GLOB)
sqldbs = [SQLiteRankingDatabase(fname=fname, name=name(fname), nomenclature="MGI") for fname in sqldb_fnames]

In [11]:
len(sqldbs)

6

In [12]:
memdb = MemoryDecorator(dbs[0])

In [13]:
len(memdb._df.index)

20003

The number of features has an impact on the time to complete a module2regulome transformation, i.e. best times are 1.5 seconds for a 24k feature database while they are 1.1 seconds for a 20k feature database.

### Load motif annotations

In [14]:
motif_annotations = load_motif_annotations(MOTIF_ANNOTATIONS_FNAME)

In [15]:
motif_annotations.head()

,,MotifSimilarityQvalue,OrthologousIdentity,Annotation
TF,MotifID,,,
Hoxa9,bergman__Abd-B,0.0006,1.000000,gene is annotated for similar motif cisbp__M10...
Zfp128,bergman__Aef1,0.0000,0.220264,motif is annotated for orthologous gene FBgn00...
Zfp853,bergman__Cf2,0.0000,0.166667,motif is annotated for orthologous gene FBgn00...
Nr1h2,bergman__EcR_usp,0.0000,0.378924,gene is orthologous to FBgn0000546 in D. melan...
Nr1h3,bergman__EcR_usp,0.0000,0.408989,gene is orthologous to FBgn0000546 in D. melan...


### Single-thread pipeline

Before scaling it via dask to work on the full combinatorial space of databases x modules.

_IMPORTANT:_ These times are for the 24k feature databases.

In [15]:
%lprun -f module2regulome list((idx, module2regulome(dbs[0], module, motif_annotations, module2features_func=module2features_bincount_impl)) for idx, module in enumerate(modules[0:25]))

In [15]:
%%snakeviz
regulomes = list((idx, module2regulome(dbs[0], module, motif_annotations, module2features_func=module2features_bincount_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmptk3z6m2i'. 


1. General performance is 78s for executing `module2regulome` 25 times.
1. 79% of time is spent at `recovery` and 18% at `db.load`.

#### SQLite-based storage + bincount implementation

In [18]:
%lprun -f module2regulome list((idx, module2regulome(sqldbs[0], module, motif_annotations, module2features_func=module2features_bincount_impl)) for idx, module in enumerate(modules[0:25]))

In [19]:
%%snakeviz
regulomes = list((idx, module2regulome(sqldbs[0], module, motif_annotations, module2features_func=module2features_bincount_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmpzsg2nvjf'. 


1. General performance is 83s for executing `module2regulome` 25 times.
1. 42% of time is spent at `recovery` and 56% at `db.load`.

#### In-memory database + bincount implementation

In [16]:
%lprun -f module2regulome list((idx, module2regulome(memdb, module, motif_annotations, module2features_func=module2features_bincount_impl)) for idx, module in enumerate(modules[0:25]))

In [23]:
%%snakeviz
regulomes = list((idx, module2regulome(memdb, module, motif_annotations, module2features_func=module2features_bincount_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmpfwp_6eat'. 


1. General performance is 78s for executing `module2regulome` 25 times.
1. 89% of time is spent at `recovery` and 8.5% at `db.load`.

#### In-memory database + bincount implementation: assess effect of reducing rank_threshold parameter

In [21]:
%lprun -f module2regulome list((idx, module2regulome(memdb, module, motif_annotations, auc_threshold=0.01, rank_threshold=750, module2features_func=module2features_bincount_impl)) for idx, module in enumerate(modules[0:25]))

1. General performance is 69s for executing `module2regulome` 25 times.
1. 93% of time is spent at `recovery` and 4.4% at `db.load`.

#### Feather-based storage + numba implementation

In [18]:
%%snakeviz
regulomes = list((idx, module2regulome(dbs[0], module, motif_annotations, module2features_func=module2features_numba_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmpslwt7n0d'. 


1. General performance is 49s for executing `module2regulome` 25 times.
1. 47% of time is spent at `recovery` and 24% at `db.load`.

#### Approach combining all potential improvements (in-memory database, auc-only calculation to assess enriched features and numba JIT implementation).

In [17]:
%%snakeviz
regulomes = list((idx, module2regulome(memdb, module, motif_annotations, module2features_func=module2features_numba_impl)) for idx, module in enumerate(modules[0:25]))

 
*** Profile stats marshalled to file '/var/folders/cj/xhw0rd3s7hg5k4p78t4s3hph0000gn/T/tmp1h5b9v30'. 


1. General performance is 39s for executing `module2regulome` 25 times.
1. 81% of time is spent at `recovery`

### Parallelized pipeline

#### Python multiprocessing implementation (db-dedicated workers using in memory copy + numba implementation of auc calculation).

Loading the database is also part of the overall timing. This will however dwarf when the number of modules increases.

In [16]:
%%timeit -n1 -r1 -o -q
regulomes = derive_regulomes(dbs[0:2], modules[0:50], MOTIF_ANNOTATIONS_FNAME, num_workers=4)
print(len(regulomes))

Using 4 workers.
2018-02-11 16:36:23.768052 - Worker mm9-500bp-upstream-10species(1): database loaded in memory.
2018-02-11 16:36:23.793022 - Worker mm9-500bp-upstream-10species(2): database loaded in memory.
2018-02-11 16:36:24.668392 - Worker mm9-tss-centered-5kb-10species(1): database loaded in memory.
2018-02-11 16:36:24.675245 - Worker mm9-tss-centered-5kb-10species(2): database loaded in memory.
2018-02-11 16:36:25.033397 - Worker mm9-500bp-upstream-10species(1): motif annotations loaded in memory.
2018-02-11 16:36:25.033497 - Worker mm9-500bp-upstream-10species(2): motif annotations loaded in memory.
2018-02-11 16:36:25.828411 - Worker mm9-tss-centered-5kb-10species(2): motif annotations loaded in memory.
2018-02-11 16:36:25.828691 - Worker mm9-tss-centered-5kb-10species(1): motif annotations loaded in memory.
2018-02-11 16:37:03.543603 - Worker mm9-500bp-upstream-10species(1): All regulomes derived.
2018-02-11 16:37:03.555085 - Worker mm9-500bp-upstream-10species(1): Done.
2018

<TimeitResult : 2min 40s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)>

In [17]:
df = derive_regulomes(dbs[0:2], modules[0:50], MOTIF_ANNOTATIONS_FNAME, output="df", num_workers=6)

Using 6 workers.
2018-02-11 16:40:19.958670 - Worker mm9-500bp-upstream-10species(3): database loaded in memory.
2018-02-11 16:40:20.080826 - Worker mm9-500bp-upstream-10species(1): database loaded in memory.
2018-02-11 16:40:20.135994 - Worker mm9-500bp-upstream-10species(2): database loaded in memory.
2018-02-11 16:40:21.332779 - Worker mm9-tss-centered-5kb-10species(3): database loaded in memory.
2018-02-11 16:40:21.403229 - Worker mm9-tss-centered-5kb-10species(2): database loaded in memory.
2018-02-11 16:40:21.463122 - Worker mm9-tss-centered-5kb-10species(1): database loaded in memory.
2018-02-11 16:40:21.731395 - Worker mm9-500bp-upstream-10species(1): motif annotations loaded in memory.
2018-02-11 16:40:21.732508 - Worker mm9-500bp-upstream-10species(2): motif annotations loaded in memory.
2018-02-11 16:40:21.733145 - Worker mm9-500bp-upstream-10species(3): motif annotations loaded in memory.
2018-02-11 16:40:22.892675 - Worker mm9-tss-centered-5kb-10species(3): motif annotatio

In [18]:
len(df)

207

In [19]:
df.head()

Enrichment                                  \
                                    AUC       NES MotifSimilarityQvalue   
TF   MotifID                                                              
Atf4 transfac_pro__M01186      0.044341  3.032623              0.000206   
     transfac_public__M00178   0.044461  3.053414              0.000000   
     cisbp__M3089              0.044916  3.132418              0.000003   
     transfac_public__M00177   0.044775  3.107969              0.000003   
     transfac_pro__M01861      0.044667  3.089216              0.000883   

                                                  \
                             OrthologousIdentity   
TF   MotifID                                       
Atf4 transfac_pro__M01186                    1.0   
     transfac_public__M00178                 1.0   
     cisbp__M3089                            1.0   
     transfac_public__M00177                 1.0   
     transfac_pro__M01861                    1.0   

                                                                                 \
                                                                     Annotation   
TF   MotifID                                                                      
Atf4 transfac_pro__M01186     gene is annotated for similar motif transfac_p...   
     transfac_public__M00178  gene is annotated for similar motif swissregul...   
     cisbp__M3089             gene is annotated for similar motif swissregul...   
     transfac_public__M00177  gene is annotated for similar motif swissregul...   
     transfac_pro__M01861     gene is annotated for similar motif transfac_p...   

                                                                                 \
                                                                        Context   
TF   MotifID                                                                      
Atf4 transfac_pro__M01186     (target weight >= 0.001, mm9-500bp-upstream-10...   
     transfac_public__M00178  (target weight >= 0.001, mm9-500bp-upstream-10...   
     cisbp__M3089             (target weight >= 0.001, mm9-500bp-upstream-10...   
     transfac_public__M00177  (target weight >= 0.001, mm9-500bp-upstream-10...   
     transfac_pro__M01861     (target weight >= 0.001, mm9-500bp-upstream-10...   

                                                                                 
                                                                    TargetGenes  
TF   MotifID                                                                     
Atf4 transfac_pro__M01186     [(Dusp1, 1.0), (Lrrtm2, 1.0), (Npas4, 1.0), (T...  
     transfac_public__M00178  [(Braf, 1.0), (Fosb, 1.0), (Vps37b, 1.0), (Rfx...  
     cisbp__M3089             [(Braf, 1.0), (Fosb, 1.0), (Ube2h, 1.0), (Egr4...  
     transfac_public__M00177  [(Braf, 1.0), (Fosb, 1.0), (Vps37b, 1.0), (Rfx...  
     transfac_pro__M01861     [(Braf, 1.0), (Tead1, 1.0), (Tpm4, 1.0), (Rfx1...

#### Dask framework using multiprocessing

In [16]:
with ProgressBar():
    with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
        regulomes = derive_regulomes(dbs[0:2], modules[0:50], MOTIF_ANNOTATIONS_FNAME,
                                     client_or_address="dask_multiprocessing", module_chunksize=8)

[########################################] | 100% Completed |  3min  8.6s


In [17]:
len(regulomes)

14

In [18]:
visualize([prof, rprof, cprof])

Column(id='91d1d41b-bdae-4402-8dec-5f1af6ff97bc', ...)

#### Dask framework with custom client

In [15]:
local_cluster = LocalCluster(n_workers=6, 
                             threads_per_worker=1)

custom_client = Client(local_cluster)

In [16]:
custom_client

Client Scheduler: tcp://127.0.0.1:51083 Dashboard: http://127.0.0.1:8787,Cluster Workers: 6 Cores: 6 Memory: 12.88 GB


In [17]:
regulomes = derive_regulomes(dbs[0:2],
                             modules[0:50],
                             MOTIF_ANNOTATIONS_FNAME,
                             client_or_address=custom_client)

In [19]:
regulomes

<Future: status: finished, type: list, key: list_of_filter-420560af-d732-480f-b278-68c9b5468ba1>

In [21]:
regulomes.result()

[Regulome(name='Regulome for Arntl', nomenclature='MGI', gene2weights=<frozendict {'Zfp668': 1.0, 'Bag6': 1.0, 'Rnf167': 1.0, 'Hmg20a': 1.0, 'Cog3': 1.0, 'Snrpb': 1.0, 'Pafah1b2': 1.0, 'Eif3i': 1.0, 'Vps52': 1.0, 'Trmt112': 1.0, 'Psma6': 1.0, 'Dnaja2': 1.0, 'Cdk13': 1.0, 'Cpsf3': 1.0, 'Cstf2t': 1.0, 'Ncstn': 1.0, 'Fam188a': 1.0, 'Tmem161b': 1.0, 'Emg1': 1.0, 'Mtmr4': 1.0, 'Cap1': 1.0, 'Epn1': 1.0, 'Ttpal': 1.0, 'Nipbl': 1.0, 'Zfp410': 1.0, 'Krr1': 1.0, 'Rpl37': 1.0, 'Dalrd3': 1.0, 'Ebna1bp2': 1.0, 'Lrrc49': 1.0, 'Pick1': 1.0, 'Ngrn': 1.0, 'Uggt1': 1.0, 'Psmd13': 1.0, 'Zfp384': 1.0, 'Sptlc2': 1.0, 'Gatad1': 1.0, 'B3gat3': 1.0, 'Tmcc1': 1.0, 'Ppme1': 1.0, 'Rhot2': 1.0, 'Fbxw2': 1.0, 'Dctn5': 1.0, 'Ndufs3': 1.0, 'Pgap3': 1.0, 'Trim46': 1.0, 'Pfn1': 1.0, 'Lyrm1': 1.0, 'Ddost': 1.0, 'Ovca2': 1.0, 'Clasp1': 1.0, 'Vma21': 1.0, 'Zfp622': 1.0, 'Exoc1': 1.0, 'Tomm6': 1.0, 'Zfp3': 1.0, 'Erh': 1.0, 'Txnl4b': 1.0, 'Prpf19': 1.0, 'Ubl7': 1.0, '1110001J03Rik': 1.0, 'Lsm1': 1.0, 'Fmr1': 1.0, 'Rars': 1

In [22]:
custom_client.close()
local_cluster.close()